In [1]:
from dotenv import load_dotenv
from portia.cli import CLIExecutionHooks
from portia import *
import requests
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Generic, TypeVar, List, ClassVar, Dict
from notion_client import Client
import os
load_dotenv(override=True)

# Fetch the Notion API key
notion_api_key = os.getenv("NOTION_API_KEY")
notion_parent_id = os.getenv("NOTION_PARENT_ID")

youtube_api_key = os.getenv("GOOGLE_API_KEY")

# Initialize the Notion client
notion = Client(auth=notion_api_key)


In [2]:
class YoutubeToolSchema(BaseModel): 

    """Input Schema for YoutubeTool."""
    topics: List[Dict[str, str]] = Field(description="The topics and page ids.")
    max_results: int = Field(default=3, description="The maximum number of results to return.")

class YoutubeTool(Tool[None]):

    """ Ask the user if they would like video suggestions to supplement their learning"""

    id: ClassVar[str] = "youtube_tool"
    name: ClassVar[str] = "YouTube Tool"
    description: ClassVar[str] = "A tool to find YouTube videos for given topics, and link them to Notion."
    args_schema = YoutubeToolSchema
    output_schema: ClassVar[tuple[str, str]] = (
        "None",
        "This tool does not return anything."
    )

    def run(self, context: ToolRunContext, topic_dict: List[Dict[str, str]], max_results: int) -> str:

        if not youtube_api_key or not notion_api_key: 
            raise EnvironmentError("Missing YOUTUBE_API_KEY or NOTION_API_KEY in environment.")
    
        for page in topic_dict: 

            topic = page['topic']
            page_id = page['page_id']

            query = f"Explain {topic}"
            url = (
                f'https://www.googleapis.com/youtube/v3/search?part=snippet&type=video'
                f'&q={query}&maxResults={max_results}&key={youtube_api_key}'
            )
            response = requests.get(url).json()
            if "items" not in response: 
                print("Error: No items found in the response.")
                continue 
                
            for item in response["items"]:
                video_id = item["id"]["videoId"]
                title = item["snippet"]["title"]
                description = item["snippet"]["description"]
                channel = item["snippet"]["channelTitle"]
                video_url = f"https://www.youtube.com/watch?v={video_id}"
            
            notion.blocks.children.append(
                    block_id=page_id,
                    children=[
                        {
                            "object": "block",
                            "type": "paragraph",
                            "paragraph": {
                                "rich_text": [
                                    {
                                        "type": "text",
                                        "text": {
                                            "content": f"{title} by {channel} — ",
                                        }
                                    },
                                    {
                                        "type": "text",
                                        "text": {
                                            "content": "Watch on YouTube",
                                            "link": {"url": video_url}
                                        }
                                    }
                                ]
                            }
                        },
                        {
                            "object": "block",
                            "type": "quote",
                            "quote": {
                                "rich_text": [
                                    {
                                        "type": "text",
                                        "text": {"content": description[:200] + "..."}
                                    }
                                ]
                            }
                        }
                    ]
                )

        return